In [19]:
from bs4 import BeautifulSoup

import requests

#first loop open the big page with url
#second loop find all links to a place 

url = "www.yelp.co.uk/search?find_loc=praha&cflt=restaurants"

r  = requests.get("http://" +url)

data = r.text

soup = BeautifulSoup(data,"lxml")

place_links = [] #store all sublinks of places

#find all links to places
for link in soup.find_all('a','biz-name'):
    place_links.append(link['href'])


place_pages = []


for place_link in place_links:
    tmpUrl = 'www.yelp.co.uk' + place_link    
    r  = requests.get("http://" +tmpUrl) #getting subpage
    data = r.text
    place_soup = BeautifulSoup(data,"lxml")
    place_pages.append(place_soup)
    
    

In [27]:
dataStorage = []

{'name':'',
 'address': '', 
 'priceRange': '', 
 'gps': [1,2], 
 'priceGroup': '', 
 'stars': '', 
 'numOfReviews': 0, 
 'web': '', 
 'numOfPhotos': '',
 'tags': [],
 'noiseLevel': '',
 'wifi': '',
 'openHours': {
        'mo' : '',
        'tu' : '',
        'we' : '',
        'th' : '',
        'fr' : '',
        'sa' : '',
        'su' : '',
    }
}

place_pages[1].find_all('address')

[<address itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress">\n<span itemprop="streetAddress">Dlouh\xe1 39</span><br/><span itemprop="postalCode">110 00</span> <span itemprop="addressLocality">Prague</span><br/>Czech Republic<br/><meta content="CZ" itemprop="addressCountry"/>\n</address>,
 <address>\n        Dlouh\xe1 39, 110 00 Praha, Czech Republic\n    </address>,
 <address>\n        Dlouh\xe1 39<br/>110 00 Prague<br/>Czech Republic\n    </address>]